In [6]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime


client = MongoClient(
    'not sharing')
db = client['WaterManagement']




def fetch_collection_data(collection_name, row_limit):
    collection = db[collection_name]
    documents = list(collection.find())
    return pd.DataFrame(documents[:row_limit])  




def determine_row_limit():
    collection_names = [
        "CustomerDetails",
        "PoliciesDetails",
        "WaterUsageOptimization",
        "Customer_ContractStatus",
        "WaterConservationPrograms",
        "ConservationParticipants",
        "ContractPolicies"
    ]


    row_limit = {}
    for collection_name in collection_names:
        collection = db[collection_name]
        count = collection.count_documents({})

        if count > 100:
            row_limit[collection_name] = 50
        elif count > 50:
            row_limit[collection_name] = 20
        else:
            row_limit[collection_name] = count

    return row_limit




def merge_collections(row_limit):
    merged_data = None


    for collection_name, limit in row_limit.items():
        df = fetch_collection_data(collection_name, limit)
        if merged_data is None:
            merged_data = df
        else:
            merged_data = pd.merge(merged_data, df, how='left', left_index=True,
                                   right_index=True, suffixes=('', f'_{collection_name}'))

    return merged_data



row_limit = determine_row_limit()


merged_data = merge_collections(row_limit)


if merged_data is not None:
    merged_data.to_csv('merged_mongo.csv', index=False)
    print("CSV file 'merged_mongo.csv' has been successfully created!")
else:
    print("No data to merge.")

CSV file 'merged_mongo.csv' has been successfully created!


In [8]:
merged_data.head()

,_id,Name,ConsumerType,Address,ContactInfo,RelevanceID,ContractID,ContractStartDate,ContractEndDate,ContractCommitment,...,ProgramName,TypeID_WaterConservationPrograms,StartDate_WaterConservationPrograms,EndDate,TargetArea,_id_ConservationParticipants,CustomerID_ConservationParticipants,ProgramID,ParticipationStartDate,BenefitsReceived
0,6794b76b1d956a8a4af23e88,Customer_1,Residential,"rua 1, Faro",+35191000001,6794b7501d956a8a4af23e83,CT00001,2023-01-11,2025-01-11,2024-01-06,...,Program_1,6794b7511d956a8a4af23e86,2023-01-31,2025-01-31,Region_1,6794b7a21d956a8a4af24126,1,2,2023-01-03,Benefits for Customer 1 in Program 2
1,6794b76b1d956a8a4af23e89,Customer_2,Residential,"rua 2, Faro",+35191000002,6794b7511d956a8a4af23e84,CT00002,2023-01-21,2025-01-21,2024-01-11,...,Program_2,6794b7511d956a8a4af23e87,2023-03-02,2025-03-02,Region_2,6794b7a21d956a8a4af24127,2,3,2023-01-05,Benefits for Customer 2 in Program 3
2,6794b76b1d956a8a4af23e8a,Customer_3,Residential,"rua 3, Faro",+35191000003,6794b7501d956a8a4af23e82,CT00003,2023-01-31,2025-01-31,2024-01-16,...,Program_3,6794b7511d956a8a4af23e85,2023-04-01,2025-04-01,Region_3,6794b7a21d956a8a4af24128,3,4,2023-01-07,Benefits for Customer 3 in Program 4
3,6794b76b1d956a8a4af23e8b,Customer_4,Residential,"rua 4, Faro",+35191000004,6794b7501d956a8a4af23e83,CT00004,2023-02-10,2025-02-10,2024-01-21,...,Program_4,6794b7511d956a8a4af23e86,2023-05-01,2025-05-01,Region_4,6794b7a21d956a8a4af24129,4,5,2023-01-09,Benefits for Customer 4 in Program 5
4,6794b76b1d956a8a4af23e8c,Customer_5,Residential,"rua 5, Faro",+35191000005,6794b7511d956a8a4af23e84,CT00005,2023-02-20,2025-02-20,2024-01-26,...,Program_5,6794b7511d956a8a4af23e87,2023-05-31,2025-05-31,Region_5,6794b7a21d956a8a4af2412a,5,6,2023-01-11,Benefits for Customer 5 in Program 6
